In [2]:
import tensorflow as tf
from math import exp

ModuleNotFoundError: No module named 'google.protobuf'

In [1]:
from tensorflow.examples.tutorials.mnist import input_data as mnist_data

# Download images and labels into mnist.test (10K images+labels) and mnist.train (60K images+labels)
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

ModuleNotFoundError: No module named 'google.protobuf'

In [13]:
# Constants
final_nodes = 10
width = 28
height = 28
area = width * height
lr = .003

In [ ]:
# Clear Tensor Names. ( Not required)
tf.reset_default_graph()

In [7]:
# Placeholders
# - Paramerters for future functions that will require data to be passed
X = tf.placeholder(tf.float32, [None, width, height, 1]) # image (28 x 28 px) / grey scale (1 channel)

Y_ = tf.placeholder(tf.float32, [None, final_nodes]) # Target results

In [8]:
# Weights and Bias (v1)
# - Variables for use by the network. These get initialized at the start
# and do not require additional data to be passed.

# 1) A two single arrays to be initialized as zero
W = tf.Variable(tf.zeros([area, final_nodes]))
B = tf.Variable(tf.zeros([final_nodes]))

In [10]:
# Model
XX = tf.reshape(X, [-1, area])
Y = tf.nn.softmax(tf.matmul(XX, W) + B)

In [11]:
# Loss Function
cross_entropy = -tf.reduce_sum(Y_ * tf.log(Y))

In [14]:
# Optimizer 
# - Backpropagation function for adjsting weights and biases
optimizer = tf.train.GradientDescentOptimizer(lr)
train_step = optimizer.minimize(cross_entropy)

In [15]:
# % of correct answers found in batch
is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [18]:
# Start Tensor Graph
# init = tf.initialize_all_variables() # Deprecated
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

In [19]:
# Training
for i in range(1000):
    # load batch of images and correct answers
    batch_X, batch_Y = mnist.train.next_batch(100)
    train_data={X: batch_X, Y_: batch_Y}

    # train
    sess.run(train_step, feed_dict=train_data)

In [22]:
# Test
test_data={X: mnist.test.images, Y_: mnist.test.labels}
a,c = sess.run([accuracy, cross_entropy], feed_dict=test_data)
print(a)

0.9171


In [ ]:
# Constants (v2)
hidden_layer = 200

In [ ]:
# Weights and Bias (v2)
# - Adding Layers: The more layers, the more filters to pick up features in your data.

W1 = tf.Variable(tf.truncated_normal([area, 200], stddev=0.1))
W2 = tf.Variable(tf.truncated_normal([200, final_nodes], stddev=0.1))

B1 = tf.Variable(tf.zeros([200]))
B2 = tf.Variable(tf.zeros([final_nodes]))

In [ ]:
# Model (v2)
# - This Model now includes an activation function.
#   Products of nodes with small values will be ignored more likely.

XX = tf.reshape(X, [-1, area])

# Activation Function
Y1 = tf.nn.sigmoid(tf.matmul(XX, W1) + B1)

# Regression function
Y = tf.nn.softmax(tf.matmul(Y1, W2) + B2)